In [121]:
import mysql.connector
import pandas as pd

# Connect to MySQL database
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Daivashree',  # replace with your password
    database='solarleads'
)

# Query data
query = "SELECT * FROM leads"
df = pd.read_sql(query, conn)

# Close connection
conn.close()




In [123]:
df.head()

,lead_id,name,age,location,property_type,owns_property,income_level,electricity_bill,contacted_before,interested,days_to_respond,bought_panel
0,6424,Shaurya Verma,34,Kedgaon,Farm,0,High,3000,0,0,22,0
1,7581,Aditya Naik,46,Bavdhan,Commercial,0,Low,8000,0,0,29,0
2,4138,Isha Verma,57,Bavdhan,Commercial,0,Low,500,0,0,15,0
3,9222,Sai Mehta,37,Kothrud,Residential,0,Low,1500,0,0,5,0
4,5228,Reyansh Verma,25,Bavdhan,Commercial,0,Medium,5000,0,0,15,0


In [125]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

df.drop(['lead_id', 'name'], axis=1, inplace=True)

for col in ['income_level', 'location', 'property_type']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])


In [129]:
df.head()

,age,location,property_type,owns_property,income_level,electricity_bill,contacted_before,interested,days_to_respond,bought_panel
0,34,5,1,0,0,3000,0,0,22,0
1,46,2,0,0,1,8000,0,0,29,0
2,57,2,0,0,1,500,0,0,15,0
3,37,6,2,0,1,1500,0,0,5,0
4,25,2,0,0,2,5000,0,0,15,0


In [131]:
# TEMPORARY: Simulate target
import numpy as np



In [133]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import joblib

X = df.drop('bought_panel', axis=1)
y = df['bought_panel']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

# Save model
joblib.dump(model, 'solar_model.pkl')


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       400

    accuracy                           1.00       400
   macro avg       1.00      1.00      1.00       400
weighted avg       1.00      1.00      1.00       400



['solar_model.pkl']

In [135]:
streamlit_code = """
import streamlit as st
import pandas as pd
import joblib

model = joblib.load('solar_model.pkl')

st.title("🌞 Solar Panel Purchase Predictor")

def get_input():
    age = st.slider("Age", 18, 80, 30)
    location = st.selectbox("Location", ['Pune', 'Kedgaon', 'Mumbai'])
    property_type = st.selectbox("Property Type", ['Farm', 'Flat', 'Villa'])
    owns_property = st.radio("Owns Property?", [0, 1])
    income_level = st.selectbox("Income Level", ['Low', 'Medium', 'High'])
    electricity_bill = st.slider("Monthly Electricity Bill (₹)", 0, 10000, 3000)
    contacted_before = st.radio("Contacted Before?", [0, 1])
    interested = st.radio("Interested?", [0, 1])
    days_to_respond = st.slider("Days to Respond", 0, 30, 5)

    location_map = {'Kedgaon': 0, 'Mumbai': 1, 'Pune': 2}
    property_map = {'Farm': 0, 'Flat': 1, 'Villa': 2}
    income_map = {'Low': 0, 'Medium': 1, 'High': 2}

    data = pd.DataFrame([[
        age,
        location_map[location],
        property_map[property_type],
        owns_property,
        income_map[income_level],
        electricity_bill,
        contacted_before,
        interested,
        days_to_respond
    ]], columns=['age', 'location', 'property_type', 'owns_property', 'income_level',
                 'electricity_bill', 'contacted_before', 'interested', 'days_to_respond'])

    return data

input_df = get_input()

if st.button("Predict"):
    prediction = model.predict(input_df)[0]
    if prediction == 1:
        st.success("✅ Likely to buy solar panels!")
    else:
        st.warning("❌ Not likely to buy solar panels.")
"""

with open("app.py", "w") as file:
    file.write(streamlit_code)

print("✅ app.py created successfully!")


✅ app.py created successfully!
